In [1]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_groq import ChatGroq  
import os
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [2]:
DATA_PATH='Data/'
def load_pdf_files(data):
    loader=DirectoryLoader(data,
                           glob='*.pdf',
                           loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents

documents=load_pdf_files(data=DATA_PATH)
print('Length of PDF Pages', len(documents))

Length of PDF Pages 2521


In [3]:
def create_chunks(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

text_chunks=create_chunks(extracted_data=documents)
print('Length of Text Chunks', len(text_chunks))

Length of Text Chunks 7640


In [4]:
def get_embedding_model():
    embedding_model=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embedding_model

embedding_model=get_embedding_model()

c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
DB_FAISS_PATH='vectorstore/db_faiss'
db=FAISS.from_documents(text_chunks, embedding_model)
db.save_local(DB_FAISS_PATH)

In [ ]:
GROQ_API_KEY = "your-groq-api-key"

def load_llm():
    llm = ChatGroq(
        api_key=GROQ_API_KEY,
        model_name="llama3-70b-8192", 
        temperature=0.9,
        max_tokens=512
    )
    return llm


# Step 2: Create Custom Prompt 
CUSTOM_PROMPT_TEMPLATE = """
You are BankBuddy.ai — a professional and intelligent banking assistant.

Behavior Rules:
1. If the user input contains a **banking-related query**:
   - Use only the given context — do not make up information.
   - If the answer is not found in the context, reply with:
     **"I'm sorry, I couldn’t find that information in the available context."**

2. If the user asks about you or your identity (e.g., "Who are you?", "Are you a bot?", "What is BankBuddy?") reply with:
    - "Who are you?" : I'm BankBuddy.ai, your intelligent assistant designed to help you with everything from loans and credit cards to accounts, KYC, and banking policies.
    - "Are you a bot?" : Yes, your intelligent assistant designed to help you with everything from loans and credit cards to accounts etc..
    - "What is BankBuddy" : BankBuddy.ai is a AI assistant designed to help with bank related queries.

Maintain a respectful and professional tone. Keep responses clear, concise, and formatted properly.

---

📄 Context:
{context}

👤 User:
{question}

💬 BankBuddy.ai:


        """

def set_custom_prompt(custom_prompt_template):
    prompt = PromptTemplate(
        template=custom_prompt_template,
        input_variables=["context", "question"]
    )
    return prompt

In [7]:
DB_FAISS_PATH = 'vectorstore/db_faiss'
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = FAISS.load_local(DB_FAISS_PATH, embedding_model, allow_dangerous_deserialization=True)

In [8]:
qa_chain = RetrievalQA.from_chain_type(
    llm=load_llm(),
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": set_custom_prompt(CUSTOM_PROMPT_TEMPLATE)}
)

In [9]:
user_query = input("Write Query Here: ")
response = qa_chain.invoke({"query": user_query})

print("\nRESULT:\n", response["result"])
print("\nSOURCE DOCUMENTS:\n", response["source_documents"])


RESULT:
 According to Article 13, when dealing with the Insurance Authority (IA) and other official agencies, the Company is required to:

1. Deal with transparency and professionalism.
2. Provide any data or information required by the Director General concerning insurance applications rejected by the Company or policies it has declined to renew, along with a statement of such rejection or non-renewal.
3. Reply to inquiries by the IA regarding any complaint submitted to and notified to the Company within an appropriate period to be set by the IA.

These requirements ensure that the Company maintains transparency and professionalism in its dealings with the IA and other official agencies.

SOURCE DOCUMENTS:
 [Document(id='b5629240-b738-4298-ad64-b858f6a601bc', metadata={'producer': 'Adobe PDF Library 16.0', 'creator': 'Adobe InDesign 16.4 (Windows)', 'creationdate': '2023-03-26T02:15:27+05:30', 'moddate': '2023-03-26T02:15:29+05:30', 'trapped': '/False', 'source': 'Data\\CBUAE_EN_156_